In [ ]:
!pip install transformers


     |████████████████████████████████| 3.4 MB 12.7 MB/s 
     |████████████████████████████████| 895 kB 49.3 MB/s 
     |████████████████████████████████| 596 kB 47.6 MB/s 
     |████████████████████████████████| 3.3 MB 40.7 MB/s 
     |████████████████████████████████| 61 kB 309 kB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


## Import modules

In [ ]:
import pandas as pd
import numpy as np
import torch
from transformers import BertForQuestionAnswering
from transformers import BertTokenizer

## Load data from stanford website

In [ ]:
data = pd.read_json('http://downloads.cs.stanford.edu/nlp/data/coqa/coqa-train-v1.0.json')
data.head()


,version,data
0,1,"{'source': 'wikipedia', 'id': '3zotghdk5ibi9ce..."
1,1,"{'source': 'cnn', 'id': '3wj1oxy92agboo5nlq4r7..."
2,1,"{'source': 'gutenberg', 'id': '3bdcf01ogxu7zdn..."
3,1,"{'source': 'cnn', 'id': '3ewijtffvo7wwchw6rtya..."
4,1,"{'source': 'gutenberg', 'id': '3urfvvm165iantk..."


In [ ]:
del data["version"]

In [ ]:
data.head(2)

,data
0,"{'source': 'wikipedia', 'id': '3zotghdk5ibi9ce..."
1,"{'source': 'cnn', 'id': '3wj1oxy92agboo5nlq4r7..."


In [ ]:
#Required columns in our dataframe
cols = ["text","question","answer"] #list of lists to create our dataframe
comp_list = []
for index, row in data.iterrows():
    for i in range(len(row["data"]["questions"])):
        temp_list = []
        temp_list.append(row["data"]["story"])
        temp_list.append(row["data"]["questions"][i]["input_text"])
        temp_list.append(row["data"]["answers"][i]["input_text"])
        comp_list.append(temp_list)

new_df = pd.DataFrame(comp_list, columns=cols) #saving the dataframe to csv file for further loading
new_df.to_csv("CoQA_data.csv", index=False)

In [ ]:
qaData = pd.read_csv("CoQA_data.csv")

In [ ]:
qaData.head(5)

,text,question,answer
0,"The Vatican Apostolic Library (), more commonl...",When was the Vat formally opened?,It was formally established in 1475
1,"The Vatican Apostolic Library (), more commonl...",what is the library for?,research
2,"The Vatican Apostolic Library (), more commonl...",for what subjects?,"history, and law"
3,"The Vatican Apostolic Library (), more commonl...",and?,"philosophy, science and theology"
4,"The Vatican Apostolic Library (), more commonl...",what was started in 2014?,a project


In [ ]:
qaData.tail(5)

,text,question,answer
108642,(CNN) -- Cristiano Ronaldo provided the perfec...,Who was a sub?,Xabi Alonso
108643,(CNN) -- Cristiano Ronaldo provided the perfec...,Was it his first game this year?,Yes
108644,(CNN) -- Cristiano Ronaldo provided the perfec...,What position did the team reach?,third
108645,(CNN) -- Cristiano Ronaldo provided the perfec...,Who was ahead of them?,Barca.
108646,(CNN) -- Cristiano Ronaldo provided the perfec...,By how much?,six points


In [ ]:
print("Number of question and answers: ", len(qaData))

Number of question and answers:  108647


In [ ]:
model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

Downloading:   0%|          | 0.00/443 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.25G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

In [ ]:
random_no = np.random.randint(0,len(data))

question = qaData["question"][random_no]
text = qaData["text"][random_no]

In [ ]:
input_ids = tokenizer.encode(question,text)
print("Input has a total of {} tokens.".format(len(input_ids)))

Input has a total of 365 tokens.


In [ ]:
tokens = tokenizer.convert_ids_to_tokens(input_ids)

for token,id in zip(tokens,input_ids):
    print('{:8}{:8,}'.format(token,id))

[CLS]        101
what       2,054
did        2,106
love       2,293
tell       2,425
them       2,068
to         2,000
buy        4,965
and        1,998
steal      8,954
?          1,029
[SEP]        102
(          1,006
cnn       13,229
)          1,007
-          1,011
-          1,011
a          1,037
federal    2,976
jury       6,467
convicted   7,979
a          1,037
california   2,662
man        2,158
monday     6,928
in         1,999
a          1,037
case       2,553
in         1,999
which      2,029
prosecutors  19,608
say        2,360
he         2,002
convinced   6,427
a          1,037
woman      2,450
to         2,000
bomb       5,968
a          1,037
federal    2,976
courthouse  10,816
so         2,061
he         2,002
could      2,071
turn       2,735
her        2,014
and        1,998
others     2,500
involved   2,920
the        1,996
scheme     5,679
in         1,999
to         2,000
authorities   4,614
,          1,010
and        1,998
collect    8,145
reward    10,377
mo

In [ ]:
sep_idx = input_ids.index(tokenizer.sep_token_id)
print("SEP token index: ", sep_idx)

#number of tokens in segment A (question) - this will be one more than the sep_idx as the index in Python starts from 0
num_seg_a = sep_idx+1
print("Number of tokens in segment A: ", num_seg_a)

num_seg_b = len(input_ids) - num_seg_a
print("Number of tokens in segment B: ", num_seg_b)

segment_ids = [0]*num_seg_a + [1]*num_seg_b

assert len(segment_ids) == len(input_ids)

SEP token index:  11
Number of tokens in segment A:  12
Number of tokens in segment B:  353


In [ ]:
output = model(torch.tensor([input_ids]))
token_type = torch.tensor([segment_ids])

In [ ]:
answer_start = torch.argmax(output.start_logits)
answer_end = torch.argmax(output.end_logits)

if answer_end >= answer_start:
  answer = " ".join(tokens[answer_start:answer_end + 1])
else:
  print("Unable to fin the answer to question")

print("\nQuestion:\n{}".format(question.capitalize()))
print("\nAnswer:\n{}.".format(answer.capitalize()))


Question:
What did love tell them to buy and steal?

Answer:
Bomb - making materials.


In [ ]:
answer = tokens[answer_start]

for i in range(answer_start+1,answer_end+1):
  if tokens[i][0:2] == "##":
    answer += tokens[i][2]
  else:
    answer += " " + tokens[i]

In [ ]:
print(answer)

bomb - making materials


In [ ]:
def question_answer(question, text):
    
    #tokenize question and text as a pair
    input_ids = tokenizer.encode(question, text)
    
    #string version of tokenized ids
    tokens = tokenizer.convert_ids_to_tokens(input_ids)
    
    #segment IDs
    #first occurence of [SEP] token
    sep_idx = input_ids.index(tokenizer.sep_token_id)    #number of tokens in segment A (question)
    num_seg_a = sep_idx+1    #number of tokens in segment B (text)
    num_seg_b = len(input_ids) - num_seg_a
    
    #list of 0s and 1s for segment embeddings
    segment_ids = [0]*num_seg_a + [1]*num_seg_b    
    
    assert len(segment_ids) == len(input_ids)
    
    #model output using input_ids and segment_ids
    output = model(torch.tensor([input_ids]), token_type_ids=torch.tensor([segment_ids]))
    
    #reconstructing the answer
    answer_start = torch.argmax(output.start_logits)
    answer_end = torch.argmax(output.end_logits)    
    
    if answer_end >= answer_start:
        answer = tokens[answer_start]
        for i in range(answer_start+1, answer_end+1):
            if tokens[i][0:2] == "##":
                answer += tokens[i][2:]
            else:
                answer += " " + tokens[i]
                
    if answer.startswith("[CLS]"):
        answer = "Unable to find the answer to your question."
    
    print("\nPredicted answer:\n{}".format(answer.capitalize()))

In [ ]:
text = """New York (CNN) -- More than 80 Michael Jackson collectibles -- including the late pop star's famous rhinestone-studded glove from a 1983 performance -- were auctioned off Saturday, reaping a total $2 million. Profits from the auction at the Hard Rock Cafe in New York's Times Square crushed pre-sale expectations of only $120,000 in sales. The highly prized memorabilia, which included items spanning the many stages of Jackson's career, came from more than 30 fans, associates and family members, who contacted Julien's Auctions to sell their gifts and mementos of the singer. Jackson's flashy glove was the big-ticket item of the night, fetching $420,000 from a buyer in Hong Kong, China. Jackson wore the glove at a 1983 performance during \"Motown 25,\" an NBC special where he debuted his revolutionary moonwalk. Fellow Motown star Walter \"Clyde\" Orange of the Commodores, who also performed in the special 26 years ago, said he asked for Jackson's autograph at the time, but Jackson gave him the glove instead. "The legacy that [Jackson] left behind is bigger than life for me,\" Orange said. \"I hope that through that glove people can see what he was trying to say in his music and what he said in his music.\" Orange said he plans to give a portion of the proceeds to charity. Hoffman Ma, who bought the glove on behalf of Ponte 16 Resort in Macau, paid a 25 percent buyer's premium, which was tacked onto all final sales over $50,000. Winners of items less than $50,000 paid a 20 percent premium."""

question = "Where was the Auction held?"
question_answer(question, text)#original answer from the dataset

print("Original answer:\n" +  qaData.loc[qaData["question"] == question]["answer"].values[0])


Predicted answer:
Hard rock cafe in new york ' s times square
Original answer:
Hard Rock Cafe


In [ ]:
text = input("Please enter your text: \n")
question = input("\nPlease enter your question: \n")

while True:
    question_answer(question, text)
    
    flag = True
    flag_N = False
    
    while flag:
        response = input("\nDo you want to ask another question based on this text (Y/N)? ")
        if response[0] == "Y":
            question = input("\nPlease enter your question: \n")
            flag = False
        elif response[0] == "N":
            print("\nBye!")
            flag = False
            flag_N = True
            
    if flag_N == True:
        break

Please enter your text: 
My name is Antony Prince I am a Final year student at St. Joseph’s College of Engineering pursuing Information Technology. I'm very excited about the field of website development and would welcome the opportunity to gain experience and work as part of the Thoughtworks team. Thoughtworks has always inspired in their way of working through collaboration and product-focused teams, especially with the pair programming approach. In my Pre-Final year, I worked as a Research Intern in the DigiBlitz technologies, which is dedicated to combining blockchain, IoT, and machine learning. I designed and worked on potential use cases while also maintaining the existing enterprise java websites in production. Throughout the internship, I consistently strived to develop use cases for applications that can propel digital transformation. Additionally, in my role as a freelancer, I liaised my team with clients to create websites for startups. With dynamic requirements from the cli